<a href="https://colab.research.google.com/github/nishantruwari/python-newbie/blob/main/implementing_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
data = pd.read_csv('/content/train.csv.zip')

In [ ]:
data = np.array(data)

In [ ]:
m, n = data.shape
np.random.shuffle(data)  # Shuffle before splitting into dev and training sets

# Split the data
data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.

# Shape of X_train (m_train examples)
_, m_train = X_train.shape

In [ ]:
def init_params():
    W1 = np.random.randn(128, 784) * 0.01  # 128 neurons in the first hidden layer
    b1 = np.zeros((128, 1))
    W2 = np.random.randn(64, 128) * 0.01  # 64 neurons in the second hidden layer
    b2 = np.zeros((64, 1))
    W3 = np.random.randn(10, 64) * 0.01   # 10 outputs (one for each digit)
    b3 = np.zeros((10, 1))
    return W1, b1, W2, b2, W3, b3


In [ ]:
def leaky_relu(x, epsilon=1-8):
# Step 1: Normalize x using Gaussian scaling
  scaled_x = x / np.sqrt(x**2 + epsilon)
# Step 2: Apply GLT-inspired transformation
  result = np.where(
      scaled_x > 0,
      scaled_x* x=,
       scaled_x
# Non-linear operation for positive inputs
# Normalized behavior for negative inputs
return
result
def leaky_relu_deriv(x, epsilon=1e-8):
    # Step 1: Normalize x using Gaussian scaling
    scaled_x = x / np.sqrt(x**2 + epsilon)

    # Step 2: Compute the derivative piecewise
    derivative = np.where(
        scaled_x > 0,
        1,  # Derivative of (scaled_x - floor(scaled_x)) is 1 for positive inputs
        (1 - np.abs(scaled_x)) / ((np.abs(scaled_x) + 1)**2)  # Derivative for negative inputs
    )
    return derivative


In [ ]:
def softmax(Z):
    exp_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True))  # Avoid overflow
    return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)

In [ ]:
def forward_prop(W1, b1, W2, b2, W3, b3, X):
    Z1 = W1.dot(X) + b1
    A1 = leaky_relu(Z1)  # Using Leaky ReLU here
    Z2 = W2.dot(A1) + b2
    A2 = leaky_relu(Z2)  # Second hidden layer with Leaky ReLU
    Z3 = W3.dot(A2) + b3
    A3 = softmax(Z3)     # Output layer
    return Z1, A1, Z2, A2, Z3, A3

In [ ]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    return one_hot_Y.T

In [ ]:
def backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y, m_train):
    one_hot_Y = one_hot(Y)

    # Compute derivatives for output layer
    dZ3 = A3 - one_hot_Y
    dW3 = (1 / m_train) * dZ3.dot(A2.T)
    db3 = (1 / m_train) * np.sum(dZ3, axis=1, keepdims=True)

    # Compute derivatives for second hidden layer
    dZ2 = W3.T.dot(dZ3) * leaky_relu_deriv(Z2)
    dW2 = (1 / m_train) * dZ2.dot(A1.T)
    db2 = (1 / m_train) * np.sum(dZ2, axis=1, keepdims=True)

    # Compute derivatives for first hidden layer
    dZ1 = W2.T.dot(dZ2) * leaky_relu_deriv(Z1)
    dW1 = (1 / m_train) * dZ1.dot(X.T)
    db1 = (1 / m_train) * np.sum(dZ1, axis=1, keepdims=True)

    return dW1, db1, dW2, db2, dW3, db3


In [ ]:
def update_params_with_adam(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3,
                            mW1, mb1, mW2, mb2, mW3, mb3, vW1, vb1, vW2, vb2, vW3, vb3,
                            alpha, beta1, beta2, epsilon, t):
    # Update moving averages of gradients (first moment)
    mW1 = beta1 * mW1 + (1 - beta1) * dW1
    mb1 = beta1 * mb1 + (1 - beta1) * db1
    mW2 = beta1 * mW2 + (1 - beta1) * dW2
    mb2 = beta1 * mb2 + (1 - beta1) * db2
    mW3 = beta1 * mW3 + (1 - beta1) * dW3
    mb3 = beta1 * mb3 + (1 - beta1) * db3

    # Update moving averages of squared gradients (second moment)
    vW1 = beta2 * vW1 + (1 - beta2) * (dW1**2)
    vb1 = beta2 * vb1 + (1 - beta2) * (db1**2)
    vW2 = beta2 * vW2 + (1 - beta2) * (dW2**2)
    vb2 = beta2 * vb2 + (1 - beta2) * (db2**2)
    vW3 = beta2 * vW3 + (1 - beta2) * (dW3**2)
    vb3 = beta2 * vb3 + (1 - beta2) * (db3**2)

    # Correct bias in moving averages
    mW1_corrected = mW1 / (1 - beta1**t)
    mb1_corrected = mb1 / (1 - beta1**t)
    mW2_corrected = mW2 / (1 - beta1**t)
    mb2_corrected = mb2 / (1 - beta1**t)
    mW3_corrected = mW3 / (1 - beta1**t)
    mb3_corrected = mb3 / (1 - beta1**t)

    vW1_corrected = vW1 / (1 - beta2**t)
    vb1_corrected = vb1 / (1 - beta2**t)
    vW2_corrected = vW2 / (1 - beta2**t)
    vb2_corrected = vb2 / (1 - beta2**t)
    vW3_corrected = vW3 / (1 - beta2**t)
    vb3_corrected = vb3 / (1 - beta2**t)

    # Update parameters
    W1 -= alpha * mW1_corrected / (np.sqrt(vW1_corrected) + epsilon)
    b1 -= alpha * mb1_corrected / (np.sqrt(vb1_corrected) + epsilon)
    W2 -= alpha * mW2_corrected / (np.sqrt(vW2_corrected) + epsilon)
    b2 -= alpha * mb2_corrected / (np.sqrt(vb2_corrected) + epsilon)
    W3 -= alpha * mW3_corrected / (np.sqrt(vW3_corrected) + epsilon)
    b3 -= alpha * mb3_corrected / (np.sqrt(vb3_corrected) + epsilon)

    return W1, b1, W2, b2, W3, b3, mW1, mb1, mW2, mb2, mW3, mb3, vW1, vb1, vW2, vb2, vW3, vb3



In [ ]:
def gradient_descent(X, Y, alpha, iterations, beta1, beta2, epsilon):
    # Initialize parameters
    W1, b1, W2, b2, W3, b3 = init_params()
    mW1, mb1, mW2, mb2, mW3, mb3 = np.zeros_like(W1), np.zeros_like(b1), np.zeros_like(W2), np.zeros_like(b2), np.zeros_like(W3), np.zeros_like(b3)
    vW1, vb1, vW2, vb2, vW3, vb3 = np.zeros_like(W1), np.zeros_like(b1), np.zeros_like(W2), np.zeros_like(b2), np.zeros_like(W3), np.zeros_like(b3)

    for t in range(1, iterations + 1):
        # Forward propagation
        Z1, A1, Z2, A2, Z3, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)

        # Backward propagation
        dW1, db1, dW2, db2, dW3, db3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y, X.shape[1])

        # Update parameters using Adam
        W1, b1, W2, b2, W3, b3, mW1, mb1, mW2, mb2, mW3, mb3, vW1, vb1, vW2, vb2, vW3, vb3 = update_params_with_adam(
            W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3,
            mW1, mb1, mW2, mb2, mW3, mb3, vW1, vb1, vW2, vb2, vW3, vb3,
            alpha, beta1, beta2, epsilon, t
        )

        # Print progress
        if t % 10 == 0:
            predictions = make_predictions(X, W1, b1, W2, b2, W3, b3)
            accuracy = get_accuracy(predictions, Y)
            print(f"Iteration {t}: Accuracy = {accuracy:.4f}")

    return W1, b1, W2, b2, W3, b3


In [ ]:
def make_predictions(X, W1, b1, W2, b2, W3, b3):
    _, _, _, _, _, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
    predictions = get_predictions(A3)
    return predictions

In [ ]:
def get_predictions(A):
    return np.argmax(A, axis=0)


In [ ]:
def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size


In [ ]:
# Initialize parameters and train
alpha = 0.001
iterations = 200
beta1 = 0.9
beta2 = 0.999
epsilon = 1e-8

W1, b1, W2, b2, W3, b3 = gradient_descent(X_train, Y_train, alpha, iterations, beta1, beta2, epsilon)

# Evaluate on dev set
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2, W3, b3)
accuracy = get_accuracy(dev_predictions, Y_dev)
print(f"Dev Set Accuracy: {accuracy}")


Iteration 10: Accuracy = 0.4505
Iteration 20: Accuracy = 0.4467
Iteration 30: Accuracy = 0.6017
Iteration 40: Accuracy = 0.7050
Iteration 50: Accuracy = 0.7749
Iteration 60: Accuracy = 0.8234
Iteration 70: Accuracy = 0.8559
Iteration 80: Accuracy = 0.8770
Iteration 90: Accuracy = 0.8905
Iteration 100: Accuracy = 0.9009
Iteration 110: Accuracy = 0.9076
Iteration 120: Accuracy = 0.9146
Iteration 130: Accuracy = 0.9199
Iteration 140: Accuracy = 0.9247
Iteration 150: Accuracy = 0.9299
Iteration 160: Accuracy = 0.9337
Iteration 170: Accuracy = 0.9386
Iteration 180: Accuracy = 0.9421
Iteration 190: Accuracy = 0.9457
Iteration 200: Accuracy = 0.9495
Dev Set Accuracy: 0.929
